# Advanced Dataframes

## Part 3 - Reshaping
- crosstab
- pivot_table

In [2]:
import pandas as pd
import numpy as np

np.random.seed(123)

In [3]:
# Create list of values for names column.
students = ['Sally', 'Jane', 'Suzie', 'Billy', 'Ada', 'John', 'Thomas',
            'Marie', 'Albert', 'Richard', 'Isaac', 'Alan']

# Randomly generate arrays of scores for each student for each subject.
# Note that all the values need to have the same length here.
math_grades = np.random.randint(low=60, high=100, size=len(students))
english_grades = np.random.randint(low=60, high=100, size=len(students))
reading_grades = np.random.randint(low=60, high=100, size=len(students))

# Randomly generate if a student is in classroom A or classroom B
classroom = np.random.choice(['A', 'B'], len(students))

df = pd.DataFrame({'name':students,
                   'math':math_grades,
                   'reading':reading_grades,
                   'english':english_grades,
                   'room':classroom})

#calculated overall grade and letter
df['overall_grade'] = df[['math','reading','english']].mean(axis=1).round(1)
df['overall_letter'] = np.where(df.overall_grade >= 90, 'A', 
                                np.where(df.overall_grade >= 80, 'B','C'))

df

,name,math,reading,english,room,overall_grade,overall_letter
0,Sally,62,80,85,A,75.7,C
1,Jane,88,67,79,B,78.0,C
2,Suzie,94,95,74,A,87.7,B
3,Billy,98,88,96,B,94.0,A
4,Ada,77,98,92,A,89.0,B
5,John,79,93,76,B,82.7,B
6,Thomas,82,81,64,A,75.7,C
7,Marie,93,90,63,A,82.0,B
8,Albert,92,87,62,A,80.3,B
9,Richard,69,94,80,A,81.0,B


## Using `.crosstab`
count the number of occurances between each subgroup
- format: `pd.crosstab(index=_, column=_)`

#### find all counts of the room and overall letter grade

In [5]:
pd.crosstab(df.room, df.overall_letter)

overall_letter,A,B,C
room,,,
A,0,5,3
B,2,1,1


#### add total counts

In [6]:
#margins
pd.crosstab(df.room, df.overall_letter, margins=True)

overall_letter,A,B,C,All
room,,,,
A,0,5,3,8
B,2,1,1,4
All,2,6,4,12


#### give percentages instead of counts

In [9]:
#normalize
pd.crosstab(df.room, df.overall_letter, normalize='index')

overall_letter,A,B,C
room,,,
A,0.0,0.625,0.375
B,0.5,0.250,0.250


In [10]:
pd.crosstab(df.room, df.overall_letter, normalize=True)

overall_letter,A,B,C
room,,,
A,0.000000,0.416667,0.250000
B,0.166667,0.083333,0.083333


### Using `.pivot_table`

creates a summary table similar to excel pivot table 

need to include 3 of the following:
- which values will make up the rows (the index)
- which values will make up the columns
- the values we are aggregating
- an aggregation method (aggfunc); if we can omit this, and mean will be used by default

format: `pd.pivot_table(data=_, index=_, columns=_, values=_, aggfunc=_)`

#### show the average math grade by overall letter grade and room

In [12]:
pd.pivot_table(data=df, index='room', columns='overall_letter', values='math')

overall_letter,A,B,C
room,,,
A,NaN,85.0,78.666667
B,95.0,79.0,88.000000


#### show the MAX math grade by overall letter grade and room

In [15]:
pd.pivot_table(data=df, index='room', columns='overall_letter', values='math', aggfunc='max')

overall_letter,A,B,C
room,,,
A,NaN,94.0,92.0
B,98.0,79.0,88.0


### Using `.map`

Use a dictionary to assign new values based on current values

- format: `.map({'old_value':'new_value'})`

#### create new df of restuarant data

In [37]:
n = 40

df = pd.DataFrame({
    'drink': np.random.choice(['Tea', 'Water', 'Water'], n),
    'meal': np.random.choice(['Curry', 'Yakisoba Noodle', 'Pad Thai'], n),
})

df.sample(10)

,drink,meal
9,Tea,Yakisoba Noodle
7,Water,Pad Thai
31,Water,Yakisoba Noodle
6,Tea,Yakisoba Noodle
19,Water,Yakisoba Noodle
29,Water,Yakisoba Noodle
22,Tea,Yakisoba Noodle
17,Water,Pad Thai
0,Tea,Pad Thai
39,Water,Yakisoba Noodle


#### what are all unique drink values

In [39]:
df.drink.unique()

array(['Tea', 'Water'], dtype=object)

#### what are all unique meal values

In [40]:
df.meal.unique()

array(['Pad Thai', 'Curry', 'Yakisoba Noodle'], dtype=object)

#### create dictionary of prices for all current items

In [41]:
prices = {
    'Yakisoba Noodle': 9,
    'Curry': 11,
    'Pad Thai': 10,
    'Tea': 2,
    'Water': 0,
}

#### map drink prices and meal prices to create bill

In [42]:
df.drink.map(prices)

0     2
1     0
2     0
3     0
4     0
5     2
6     2
7     0
8     0
9     2
10    0
11    0
12    0
13    0
14    2
15    0
16    0
17    0
18    0
19    0
20    2
21    2
22    2
23    0
24    0
25    0
26    0
27    0
28    0
29    0
30    0
31    0
32    0
33    0
34    0
35    2
36    0
37    2
38    0
39    0
Name: drink, dtype: int64

In [43]:
df.meal.map(prices)

0     10
1     10
2     10
3     10
4     10
5     11
6      9
7     10
8     10
9      9
10     9
11     9
12     9
13    10
14     9
15     9
16    10
17    10
18    11
19     9
20    11
21    11
22     9
23    11
24    10
25     9
26    10
27    10
28    10
29     9
30     9
31     9
32    10
33    11
34    11
35    11
36    10
37     9
38    10
39     9
Name: meal, dtype: int64

#### how many orders of each combination of meal and drink?

In [45]:
df['bill'] = df.drink.map(prices) + df.meal.map(prices)
df

,drink,meal,bill
0,Tea,Pad Thai,12
1,Water,Pad Thai,10
2,Water,Pad Thai,10
3,Water,Pad Thai,10
4,Water,Pad Thai,10
5,Tea,Curry,13
6,Tea,Yakisoba Noodle,11
7,Water,Pad Thai,10
8,Water,Pad Thai,10
9,Tea,Yakisoba Noodle,11


#### what percentage of each?

In [49]:
pd.crosstab(df.meal, df.drink, normalize=True) * 100

drink,Tea,Water
meal,,
Curry,10.0,10.0
Pad Thai,2.5,40.0
Yakisoba Noodle,12.5,25.0


#### whats the average bill for each combination?

In [50]:
pd.pivot_table(df, 'bill', 'meal','drink')

drink,Tea,Water
meal,,
Curry,13,11
Pad Thai,12,10
Yakisoba Noodle,11,9


In [51]:
df.groupby(['meal','drink']).mean()

bill
meal            drink      
Curry           Tea    13.0
                Water  11.0
Pad Thai        Tea    12.0
                Water  10.0
Yakisoba Noodle Tea    11.0
                Water   9.0

#### we can also get this by using a groupby